# Graph Neural Network Model Training

This notebook demonstrates the training of a Graph Neural Network (GNN) model using the processed financial data. The steps include:
- Loading the graph data using the custom `load_data` function
- Initializing a Graph Convolutional Network (GCN) model
- Training the model while monitoring the loss over epochs
- Plotting the training loss
- Evaluating model performance on the training data

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from src.model_training import load_data, GCN

# Load the graph data from processed CSV
data, num_nodes, num_features = load_data()
print(f'Loaded graph with {num_nodes} nodes and {num_features} feature(s) per node.')

In [ ]:
# Initialize the model and optimizer
model = GCN(num_features=num_features, hidden_channels=16, num_classes=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Set training parameters
epochs = 100
loss_history = []

model.train()
for epoch in range(1, epochs + 1):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out, data.y)
    loss.backward()
    optimizer.step()
    loss_history.append(loss.item())
    if epoch % 10 == 0:
        print(f'Epoch {epoch:03d}, Loss: {loss.item():.4f}')

print('Training completed.')

In [ ]:
# Plot the training loss over epochs
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs + 1), loss_history, marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.grid(True)
plt.show()

In [ ]:
# Evaluate the model's performance on the training data
model.eval()
with torch.no_grad():
    out = model(data)
    # Get predicted classes
    pred = out.argmax(dim=1)
    # Calculate accuracy
    correct = pred.eq(data.y).sum().item()
    accuracy = correct / num_nodes
    print(f'Accuracy on training data: {accuracy * 100:.2f}%')